In [ ]:
import pandas as  pd
import os
from sklearn.preprocessing import LabelEncoder
path = '/workspace/data/Etri/processed_data'
csv_files = [file for file in os.listdir(path) if file.endswith('.csv')]
dataframes = [pd.read_csv(os.path.join(path, file)) for file in csv_files]

df = pd.concat(dataframes, ignore_index=True)

df['datetime'] = pd.to_datetime(df['datetime'])
df = df.set_index('datetime')
df.sort_index(inplace=True)
gap = 10000
now = 0 
to_path = '/workspace/data/Etri/date_continual/csv_10000'
while df.shape[0] > now:
    tmp_df = df[now:now+gap]
    tmp_df.to_csv(os.path.join(to_path,f'{now}_ETRI.csv'))
    now+=gap



## 시간별로 새롭게 들어오는 데이터 전처리


In [ ]:
import pandas as  pd
from sklearn.preprocessing import LabelEncoder
import os

pre_dir = '/workspace/data/Etri/date_continual/csv_10000/'
user_lst = os.listdir(pre_dir)
encoder = LabelEncoder()
user_lst.sort()
for user in user_lst:
    df = pd.read_csv(pre_dir+user)
    user_number = user.split('.')[0]
        
    df = df[['e4Hr__x', 'mAcc__x', 'mAcc__y', 'mAcc__z', 'e4Temp__x', 'mMag__x',
    'mMag__y', 'mMag__z', 'e4Bvp__x', 'mGps__lat', 'mGps__lon',
    'mGps__accuracy', 'mGyr__x', 'mGyr__y', 'mGyr__z', 'e4Acc__x',
    'e4Acc__y', 'e4Acc__z', 'e4Eda__x','emotionPositive', 'emotionTension']]

    col_means = df.mean()
    
    # fill missing values with the mean of the corresponding column

    # 기본 전처리 

    df = df.fillna(col_means)

    
    df['e4Acc'] = df.apply(lambda x: [x['e4Acc__x'], x['e4Acc__y'], x['e4Acc__z']], axis=1)
    df['mAcc'] = df.apply(lambda x: [x['mAcc__x'], x['mAcc__y'], x['mAcc__z']], axis=1)

    df['mGyr'] = df.apply(lambda x: [x['mGyr__x'], x['mGyr__y'], x['mGyr__z']], axis=1)
    df['mGps'] = df.apply(lambda x: [x['mGps__lat'], x['mGps__lon']], axis=1) #j, x['mGps__accuracy']
    df['mMag'] = df.apply(lambda x: [x['mMag__x'], x['mMag__y'], x['mMag__z']], axis=1)
    df['mAcc'] = df.apply(lambda x: [x['mAcc__x'], x['mAcc__y'], x['mAcc__z']], axis=1)
    df['emotionTension'] = df['emotionTension'] -1 
    # df['emotionTension'] = df['emotionTension'].apply(lambda x: 0 if x in [0, 1] else 1 if x in [2, 3, 4] else 2)
    df['tension_label'] = encoder.fit_transform(df['emotionTension'])
    print(df['tension_label'])
    break

    # df.to_pickle(f'/workspace/data/Etri/date_continual/pickle_2/new_{user_number}.pkl')
    print(user,'----'*19)
    print('total : ',df.shape)
    for i in range(1,8):
        print(f'{i} : ',df[df['emotionTension']==i].shape ,f" Size : {df[df['emotionTension']==i].shape[0] / df.shape[0]}")
        
    # print(df[df['emotionTension']==0].shape)
    # print(df[df['emotionTension']==1].shape)
    # print(df[df['emotionTension']==2].shape)
    print('====='*20)
